## Scapping de Code du travail
## Partie réglementaire (Articles R1111-1 à R8323-2)
### https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006072050/LEGISCTA000018488606/#LEGISCTA000018532586

In [26]:
import pandas as pd
import numpy as np
import dateparser
import re
import requests
from bs4 import BeautifulSoup
import os

url = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006072050/LEGISCTA000018488606/#LEGISCTA000018532586"

def process_line(text):
    text = text.replace('R. ', ' R. ')
    text = re.sub(r'([;:°])(\s*[0-9a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'\)(\s*[0-9a-zA-Z])', r') \1', text)
    text = re.sub(r'\.(?=\s*[A-Z])', '. ', text)
    return text

def find_cuts(text):
    positions = []
    for match in re.finditer(r'\. (?=[A-Z])', text):
        positions.append(match.start())
    return len(positions) + 1

def get_articles_limit(text):
    pattern = r'\((Articles? )?(R\d+-\d+)(?: à (R\d+-\d+))?\)'
    match = re.search(pattern, text)
    if match:
        begin_ = match.group(2) 
        end_ = match.group(3) if match.group(3) else begin_
        return begin_, end_
    else:
        return None, None

def reference_to_ID(article):
    base_3 = article[1:article.find('-')]
    base_1 = article[article.find('-')+1:]
    if base_1.find('-') ==-1:
        return 1000 * int(base_3) + int(base_1)
    else:
        base_float = base_1[base_1.find('-')+1:]
        base_1 = base_1[base_1.find('-')+1:]
        return 1000 * int(base_3) + int(base_1) + int(base_float)/100

        
def get_articles_info(article_):
    x = reference_to_ID(article_.replace('Article ', ''))
    df_tmp = df_chapters.copy()
    df_tmp[['Debut', 'Fin']] = df_tmp[['Debut', 'Fin']].map(reference_to_ID).astype(int)
    df_ = df_tmp[(df_tmp['Debut'] <= x) & (df_tmp['Fin'] >= x)].copy()
    tit = df_['Titre'].dropna().values[0]
    contenu_tit = df_['Contenu_Titre'].dropna().values[0]
    if len(df_['Chapitre'].dropna()) > 0:
        chap = df_['Chapitre'].dropna().values[0]
        contenu_chap = df_['Contenu_Chapitre'].dropna().values[0]
    else:
        chap = 1
        contenu_chap = 'Pas de Chapitre'
    if len(df_['Section'].dropna()) > 0:
        sec = df_['Section'].dropna().values[0]
        contenu_sec = df_['Contenu_Section'].dropna().values[0]
    else:
        sec = 1
        contenu_sec = 'Pas de titre'
    return tit, contenu_tit, chap, contenu_chap, sec, contenu_sec
    
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    df_chapters = pd.DataFrame(columns=['Titre', 'Chapitre', 'Section', 'Debut', 'Fin', 'Contenu Titre', 'Contenu Chapitre', 'Contenu Section'])
    for span in soup.find_all('span', {'data-anchor': True})[:-1]:
        txt = span.get_text(strip=True)
        beg, en = get_articles_limit(txt)
        for vocab in ['Titre', 'Chapitre', 'Section']:
            if txt[:len(vocab)] == vocab:
                globals()[f'Last_{vocab}'] = txt[:txt.find(':')-1]
                df_tmp = pd.DataFrame({
                    'Titre': None, 
                    'Chapitre': None,
                    'Section': None,
                    'Contenu Titre': None,
                    'Contenu Chapitre': None,
                    'Contenu Section': None,
                    'Debut': [beg], 
                    'Fin': [en],
                })
                df_tmp[f'Contenu_{vocab}'] = txt[txt.find(':')+1:].strip()
                df_tmp[vocab] = txt[:txt.find(':')-1]
                df_chapters = pd.concat([df_chapters, df_tmp])
    
    numbers_dict = ({'Ier': 1, 'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10, 
                    'préliminaire': 0, 'III bis': 3, 'VIII bis': 8, 'V BIS': 5, 'uniqu':0})
    # df_chapters['Titre'] = df_chapters['Titre'].map(lambda x: numbers_dict[x.replace('Titre ', '').strip()] if x else None)
    # df_chapters['Chapitre'] = df_chapters['Chapitre'].map(lambda x: numbers_dict[x.replace('Chapitre ', '').strip()] if x else None)
    # df_chapters['Section'] = df_chapters['Section'].map(lambda x: x.replace('Section ', '').strip() if x else None)  
    df_chapters['Titre'] = df_chapters['Titre'].ffill()
    
    articles_data = []
    for k, article in enumerate(soup.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))):
        article_name = article.get_text(strip=True)
        mise_en_application = article.find_next('a', href=lambda x: x and '/loda/id/' in x)
        mise_en_application_text = mise_en_application.get_text(strip=True) if mise_en_application else 'Non disponible'
        content_article = article.find_next('div', class_='content')
        if content_article:
            content = content_article.get_text(strip=True)
        else:
            content = 'Contenu non disponible'

        tit, contenu_tit, chap, contenu_chap, sec, contenu_sec = get_articles_info(article_name)
        articles_data.append({
            'N° Titre': tit,
            'Titre': contenu_tit,
            'N° Chapitre': chap,
            'Chapitre': contenu_chap,
            'N° Section': sec,
            'Section': contenu_sec,
            'Article': article_name,
            'Mise en application': mise_en_application_text,
            'Contenu': content
        })
        
    df = pd.DataFrame(articles_data)    
    df['Contenu'] = df['Contenu'].map(process_line)
    df['Nombre phrases'] = df['Contenu'].map(find_cuts)
else:
    print("Erreur lors du chargement de la page :", response.status_code)
    df['N° Titre'] = df['N° Titre'].astype(int)
    df['N° Chapitre'] = df['N° Chapitre'].astype(int)

df['Source'] = 'Etablissement recevant des travailleurs'

df['Contexte'] = ''
for i in range(len(df)):
    for j in [1, 3, 5, 6, 7]:
        if not(pd.isnull(df.loc[i, df.columns[j]])):
            df.loc[i, 'Contexte'] = df.loc[i, 'Contexte'] + str(df.loc[i, df.columns[j]]) + '.x. '

df_ert = df[['Source', 'Contexte', 'Contenu']].copy()

In [27]:
df_ert

,Source,Contexte,Contenu
0,Etablissement recevant des travailleurs,Obligations du maître d'ouvrage pour la concep...,"Les dispositions du présent titre déterminent,..."
1,Etablissement recevant des travailleurs,Obligations du maître d'ouvrage pour la concep...,"Pour l'application du présent titre, on entend..."
2,Etablissement recevant des travailleurs,Obligations du maître d'ouvrage pour la concep...,Le maître d'ouvrage élabore et transmet aux ut...
3,Etablissement recevant des travailleurs,Obligations du maître d'ouvrage pour la concep...,Le dossier de maintenance des lieux de travail...
4,Etablissement recevant des travailleurs,Obligations du maître d'ouvrage pour la concep...,Le dossier de maintenance des lieux de travail...
...,...,...,...
294,Etablissement recevant des travailleurs,Obligations de l'employeur pour l'utilisation ...,Les dispositions relatives à l'hébergement des...
295,Etablissement recevant des travailleurs,Obligation de vigilance et responsabilité des ...,Pour la mise en œuvre de l'injonction prévue à...
296,Etablissement recevant des travailleurs,Obligation de vigilance et responsabilité des ...,"Dès réception de l'injonction, l'employeur inf..."
297,Etablissement recevant des travailleurs,Obligation de vigilance et responsabilité des ...,En cas d'absence de régularisation effective d...


In [28]:
import pandas as pd
import numpy as np
import dateparser
import re
import requests
from bs4 import BeautifulSoup
import os

url = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006074096/LEGISCTA000006112857/#LEGISCTA000006112857"

def process_line(text):
    text = text.replace('R. ', ' R. ')
    text = re.sub(r'([;:°])(\s*[0-9a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'\)(\s*[0-9a-zA-Z])', r') \1', text)
    text = re.sub(r'\.(?=\s*[A-Z])', '. ', text)
    return text

def find_cuts(text):
    positions = []
    for match in re.finditer(r'\. (?=[A-Z])', text):
        positions.append(match.start())
    return len(positions) + 1

def reference_to_ID(x):
    lst = np.array(str(x).replace('*', '').replace('.', '').replace("Annexe II à l'article", '').replace("Annexe I à l'article", '').replace('Annexe III', '').replace('A', '1').strip().split('-'))
    n = 0
    if len(lst)>1:
        lst[0] = lst[0][1:]
        for i in range(len(lst)):
            if not(lst[len(lst) - i -1].isdigit()):
                print(x)
                print(lst)
            n += int(lst[len(lst) - i -1]) * 10**(3*i)
    return n

        
def get_articles_info(article_):
    x = reference_to_ID(article_.replace('Article ', ''))
    df_tmp = df_chapters.copy()
    df_ = df_tmp[(df_tmp['deb_R'] <= x) & (df_tmp['fin_R'] >= x)].copy()
    dict_var = dict({'Livre': 'liv', 'Titre': 'tit', 'Chapitre': 'chap', 'Section': 'sec', 'Sous-section': 'ssec', 'Paragraphe': 'para'})
    for item in ['Livre', 'Titre', 'Chapitre', 'Section', 'Sous-section', 'Paragraphe']:
        lst = df_[f'Contenu_{item}'].dropna().values
        if len(lst)>0:
            globals()[f'{dict_var.get(item)}'] = lst[0]
        else:
            globals()[f'{dict_var.get(item)}'] = None
    
    return liv, tit, chap, sec, ssec, para
    
response = requests.get(url)


soup = BeautifulSoup(response.content, 'html.parser')

df_chapters = pd.DataFrame(columns=['Livre', 'Titre', 'Chapitre', 'Section', 'Sous-section', 'Paragraphe', 'Debut', 'Fin'])
for span in soup.find_all('span', {'data-anchor': True})[:-1]:
    txt = span.get_text(strip=True)
    for vocab in ['Livre', 'Titre', 'Chapitre', 'Section', 'Sous-section', 'Paragraphe']:
        if txt[:len(vocab)] == vocab:
            df_tmp = pd.DataFrame(columns=['Livre', 'Titre', 'Chapitre', 'Section', 'Sous-section', 'Paragraphe', 'Debut', 'Fin'])
            df_tmp.loc[0, f'Contenu_{vocab}'] = txt[txt.find(':')+1:txt.find('(')].strip()
            df_tmp.loc[0, vocab] = txt[:txt.find(':')-1]
            if '(' and ')' in txt:
                part = txt[txt.find('(')+1:txt.find(')')]
                if 'Articles' in part:
                    df_tmp.loc[0, 'Debut'] = part[9: part.find('à')-1]
                    df_tmp.loc[0, 'Fin'] = part[part.find('à')+2:]
                elif 'Article' in part:
                    df_tmp.loc[0, 'Debut'] = part[7:]
                    df_tmp.loc[0, 'Fin'] = part[7:]
            df_chapters = pd.concat([df_chapters, df_tmp])
            
df_chapters.dropna(subset=['Debut'], inplace=True) 
df_chapters.reset_index(inplace=True, drop=True)

df_chapters['deb_R'] = df_chapters['Debut'].apply(reference_to_ID)
df_chapters['fin_R'] = df_chapters['Fin'].apply(reference_to_ID)
df_chapters.drop(columns = ['Livre', 'Titre', 'Chapitre', 'Section', 'Sous-section', 'Paragraphe', 'Debut', 'Fin'], inplace=True)
# numbers_dict = ({'IER': 1, 'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10, 'XI': 11, 'PRÉLIMINAIRE': 0,
#                  'IER BIS': 1, 'I BIS': 1, 'II BIS': 2, 'III BIS': 3, 'IV BIS': 4, 'V BIS': 5, 'VI BIS': 6, 'VII BIS': 7, 'VIII BIS': 8, 
#                  'IX BIS': 9, 'X BIS': 10, 'XI BIS': 11})
# df_chapters['Titre'] = df_chapters['Titre'].map(lambda x: numbers_dict[x.replace('Titre ', '').strip().upper()] if x else None)
# df_chapters['Titre'] = df_chapters['Titre'].map(lambda x: numbers_dict[x.replace('Titre ', '').strip().upper()] if x else None)
# df_chapters['Chapitre'] = df_chapters['Chapitre'].map(
#     lambda x: numbers_dict.get(x.replace('Chapitre ', '').strip().upper()) if x else None)
# df_chapters['Section'] = df_chapters['Section'].map(lambda x: x.replace('Section ', '').strip() if x else None)  
   
articles_data = []
for k, article in enumerate(soup.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))):
    article_name = article.get_text(strip=True)
    mise_en_application = article.find_next('a', href=lambda x: x and '/loda/id/' in x)
    mise_en_application_text = mise_en_application.get_text(strip=True) if mise_en_application else 'Non disponible'
    content_article = article.find_next('div', class_='content')
    if content_article:
        content = content_article.get_text(strip=True)
    else:
        content = 'Contenu non disponible'

    liv, tit, chap, sec, ssec, para = get_articles_info(article_name)
    articles_data.append({
        'Article': article_name,
        'Livre': liv,
        'Titre': tit,
        'Chapitre': chap,
        'Section': sec,
        'Sous-section': ssec,
        'Paragraphe': para,
        'Mise en application': mise_en_application_text,
        'Contenu': content
    })
        
    df = pd.DataFrame(articles_data)    
    df['Contenu'] = df['Contenu'].map(process_line)

df['Source'] = "Code de la construction et de l'habitation"

df['Contexte'] = ''
for i in range(len(df)):
    for j in range(8):
        if not(pd.isnull(df.loc[i, df.columns[j]])):
            df.loc[i, 'Contexte'] = df.loc[i, 'Contexte'] + str(df.loc[i, df.columns[j]]) + '.x. '

df_hab = df[['Source', 'Contexte', 'Contenu']].copy()

R. 261-23-1
[' 261' '23' '1']


In [29]:
df_hab

,Source,Contexte,Contenu
0,Code de la construction et de l'habitation,"Article R111-1.x. Construction, entretien et r...",Constituent des bâtiments d'habitation au sens...
1,Code de la construction et de l'habitation,"Article R111-2.x. Construction, entretien et r...",I. - Pour l'application des dispositions du pr...
2,Code de la construction et de l'habitation,"Article R112-1.x. Construction, entretien et r...",Pour obtenir l'attestation de respect des obje...
3,Code de la construction et de l'habitation,"Article R112-2.x. Construction, entretien et r...",Le dossier de demande de l'attestation de resp...
4,Code de la construction et de l'habitation,"Article R112-3.x. Construction, entretien et r...",I. - L'organisme tiers analyse la solution d'e...
...,...,...,...
2654,Code de la construction et de l'habitation,Article R863-13.x. Aides personnelles au logem...,Pour leur application à Saint-Pierre-et-Miquel...
2655,Code de la construction et de l'habitation,Article D863-14.x. Aides personnelles au logem...,Pour leur application à Saint-Pierre-et-Miquel...
2656,Code de la construction et de l'habitation,Article D863-15.x. Aides personnelles au logem...,Les articles D. 842-15 à D. 842-18 ne sont pas...
2657,Code de la construction et de l'habitation,Article R863-16.x. Aides personnelles au logem...,"Les articles R. 842-14, R. 843-2 à R. 843-8..."


In [30]:
def get_contenu(i):
    if df_cleaned.loc[i, 'Link URL']:
        response = requests.get(base_url + df_cleaned.loc[i, 'Link URL'][6:])
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            DD = soup.find_all('div', class_="Date")
            if len(DD)>0:
                date_arrete = DD[0].get_text(strip=True)
            else:
                date_arrete = None
            if soup.find('div', class_='CadreTxt'):
                texte = soup.find('div', class_='CadreTxt').get_text(strip=True, separator='\n').replace('\xa0', ' ').replace('\n', ' ')
            else :
                texte = None
            return date_arrete, texte
    else:
        return None, None
        
root_url = 'https://sitesecurite.com/contenu/portail/erp/erp-instructions-techniques.php'
base_url = 'https://sitesecurite.com/contenu/'

response = requests.get(root_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    accordion_data = []
    first_level_counter = 1
    accordion_containers = soup.find_all('div', class_='accordion-container')
    for container in accordion_containers:
        title = container.find('h2', class_='ArtLigneTitre').get_text(strip=True) if container.find('h2', class_='ArtLigneTitre') else "No Title"
        first_level_reference = str(first_level_counter)
        accordion_data.append({
            'Reference': first_level_reference,
            'Title': title,
            'Section Title': None,
            'Link Text': None,
            'Link URL': None
        })
        second_level_counter = 1
        container_tabs = container.find_all('div', class_='containerTab')
        for tab in container_tabs:
            full_reference = f"{first_level_reference}.{second_level_counter}"
            section_title = tab.find('div', class_='intitule').get_text(strip=True) if tab.find('div', class_='intitule') else "No Section Title"
            link_text = tab.find('div', class_='clic000').get_text(strip=True) if tab.find('div', class_='clic000') else "No Link Text"
            link = tab.find('div', class_='clic000')['onclick'] if tab.find('div', class_='clic000') else None
            if link:
                link_url = re.search(r"DoHref\('(.+?)'\)", link)
                link_url = link_url.group(1) if link_url else None
            else:
                link_url = None
            accordion_data.append({
                'Reference': full_reference,
                'Title': title,
                'Section Title': section_title,
                'Link Text': link_text,
                'Link URL': link_url
            })
            second_level_counter += 1
        first_level_counter += 1
    df = pd.DataFrame(accordion_data)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

df_cleaned = df.dropna(subset=['Link URL']).copy()
df_cleaned.reset_index(inplace=True, drop=True)

df_cleaned['Date Arrete'] = None
df_cleaned['Contenu'] = None



for i in range(len(df_cleaned)):
    date_a, txt = get_contenu(i)
    df_cleaned.loc[i, 'Contenu'] = txt
    df_cleaned.loc[i, 'Date Arrete'] = date_a

df = df_cleaned.dropna(subset=['Contenu']).copy()
df.reset_index(inplace=True, drop=True)

df['Source'] = "Réglementation sécurité et incendie. Etablissement recevant du public"

df['Contexte'] = df['Source']
for i in range(len(df)):
    for j in [1, 2, 5]:
        if not(pd.isnull(df.loc[i, df.columns[j]])):
            df.loc[i, 'Contexte'] = df.loc[i, 'Contexte'] + str(df.loc[i, df.columns[j]]) + '.x. '

df_rsi = df[['Source', 'Contexte', 'Contenu']].copy()

In [31]:
df_rsi

,Source,Contexte,Contenu
0,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,1. Objet Le chapitre IV du titre I er du livre...
1,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,2. Terminologie Pour l'application de la prése...
2,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,3. Dispositions relatives au désenfumage natur...
3,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,4. Dispositions relatives au désenfumage mécan...
4,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,5. Solutions applicables aux escaliers enclois...
5,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,6. Solutions applicables aux circulations encl...
6,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,7. Solutions applicables aux locaux accessible...
7,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,8. Prescriptions relatives aux approches d'ing...
8,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,Annexe : Détermination de la surface utile d'o...
9,Réglementation sécurité et incendie. Etablisse...,Réglementation sécurité et incendie. Etablisse...,1. Terminologie Volet : dispositif d'obturatio...


In [32]:
# url à scrapper
url = "https://www.legifrance.gouv.fr/loda/id/LEGITEXT000020303557"

# Envoyer une requête GET à la page et vérifier si elle a réussi
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    data = []
    seen_articles = set()  # nécessaire pour supprimer les doublons d articles - suite ajout section ou absence de section

    livres = soup.find_all('span', attrs={'data-anchor': True})

    for livre_element in livres:
        if 'Livre' in livre_element.text:
            text_livre = livre_element.text.strip()

            ul_titre = livre_element.find_next('ul', class_='js-child')

            if ul_titre:
                titres = ul_titre.find_all('span', attrs={'data-anchor': True})

                for titre_element in titres:
                    if 'Titre' in titre_element.text:
                        text_titre = titre_element.text.strip()

                        # Récupérer les chapitres du titre
                        ul_chapitre = titre_element.find_next('ul', class_='js-child')

                        if ul_chapitre:
                            chapitres = ul_chapitre.find_all('span', attrs={'data-anchor': True})

                            for chapitre in chapitres:
                                if 'Chapitre' in chapitre.text:
                                    chapitre_titre = chapitre.text.strip()

                                    ul_section = chapitre.find_next('ul', class_='js-child')
                                    if ul_section:
                                        sections = ul_section.find_all('span', attrs={'data-anchor': True})

                                        for section in sections:
                                            section_titre = section.text.strip()

                                            # Trouver les articles dans la section
                                            ul_articles = section.find_next('ul', class_='js-child')

                                            if ul_articles:
                                                articles = ul_articles.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))

                                                for article in articles:
                                                    article_name = article.get_text(strip=True)

                                                    if article_name in seen_articles:
                                                        continue
                                                    seen_articles.add(article_name)

                                                    mise_en_application = article.find_next('p', class_='date')
                                                    mise_en_application = mise_en_application.text.strip() if mise_en_application else "Non trouvé"

                                                    contenu = article.find_next('div', class_='content')
                                                    contenu = contenu.text.strip() if contenu else "Non trouvé"

                                                    # Ajouter des informations récupérées
                                                    data.append({
                                                        "Livre": text_livre,
                                                        "Titre": text_titre,
                                                        "Chapitre": chapitre_titre,
                                                        "Section": section_titre,
                                                        "Numéro Article": article_name,
                                                        "Mise en application": mise_en_application,
                                                        "Contenu": contenu
                                                    })

            # Récupérer les chapitres sans titres
            chapitres_sans_titres = ul_titre.find_all('span', attrs={'data-anchor': True})

            for chapitre in chapitres_sans_titres:
                if 'Chapitre' in chapitre.text:
                    chapitre_titre = chapitre.text.strip()

                    # Trouver les sections dans ces chapitres
                    ul_section = chapitre.find_next('ul', class_='js-child')
                    if ul_section:
                        sections = ul_section.find_all('span', attrs={'data-anchor': True})

                        for section in sections:
                            section_titre = section.text.strip()

                            # Trouver les articles dans la section
                            ul_articles = section.find_next('ul', class_='js-child')

                            if ul_articles:
                                articles = ul_articles.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))

                                for article in articles:
                                    article_name = article.get_text(strip=True)

                                    if article_name in seen_articles:
                                        continue
                                    seen_articles.add(article_name)

                                    mise_en_application = article.find_next('p', class_='date')
                                    mise_en_application = mise_en_application.text.strip() if mise_en_application else "Non trouvé"

                                    contenu = article.find_next('div', class_='content')
                                    contenu = contenu.text.strip() if contenu else "Non trouvé"

                                    # Ajouter les informations récupérées
                                    data.append({
                                        "Livre": text_livre,
                                        "Titre": "Non associé",
                                        "Chapitre": chapitre_titre,
                                        "Section": section_titre,
                                        "Numéro Article": article_name,
                                        "Mise en application": mise_en_application,
                                        "Contenu": contenu
                                    })

    # Ajouter les contenus non associés à des livres, titres ou chapitres
    articles_without_title = soup.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))
    for article in articles_without_title:
        if article.get_text(strip=True) not in seen_articles:
            article_name = article.get_text(strip=True)
            seen_articles.add(article_name)

            mise_en_application = article.find_next('p', class_='date')
            mise_en_application = mise_en_application.text.strip() if mise_en_application else None

            contenu = article.find_next('div', class_='content')
            contenu = contenu.text.strip() if contenu else None

            data.append({
                "Livre": "Non associé",
                "Titre": "Non associé",
                "Chapitre": "Non associé",
                "Section": "Non associé",
                "Numéro Article": article_name,
                "Mise en application": mise_en_application,
                "Contenu": contenu
            })

    df = pd.DataFrame(data)

    # Data cleaning sur les différentes colonnes
    df['Num_Livre'] = df['Livre'].apply(lambda x: x.split(':')[0].strip() if ':' in x else x)
    df['Livre'] = df['Livre'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x)

    df['Num_Titre'] = df['Titre'].apply(lambda x: x.split(':')[0].strip() if ':' in x else x)
    df['Titre'] = df['Titre'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x)

    df['Num_Chapitre'] = df['Chapitre'].apply(lambda x: x.split(':')[0].strip() if ':' in x else x)
    df['Chapitre'] = df['Chapitre'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x)
    df['Chapitre'] = df['Chapitre'].str.replace(r'\s*\(.*?\)', '', regex=True)

    df['Num_Section'] = df['Section'].apply(lambda x: x.split(':')[0].strip() if ':' in x else x)
    df['Section'] = df['Section'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x)
    df['Section'] = df['Section'].str.replace(r'\s*\(.*?\)', '', regex=True)

    df['Numéro Article'] = df['Numéro Article'].str.replace('Article ', '', regex=False)

    df['Type arrêté'] = df['Mise en application'].str.extract(r'^(.*?)\s*Arrêté')

    df['Mise en application'] = df['Mise en application'].str.replace(r'.*?\s*(Arrêté.*)', r'\1', regex=True)
    df['Mise en application'] = df['Mise en application'].str.replace(r'\n', ' ', regex=True)

    columns_order = [
        'Num_Livre',
        'Livre',
        'Num_Titre',
        'Titre',  
        'Num_Chapitre', 
        'Chapitre', 
        'Num_Section',   
        'Section', 
        'Numéro Article', 
        'Type arrêté', 
        'Mise en application', 
        'Contenu'
    ]
    df = df[columns_order]


df['Source'] = "Règlement de sécurité contre les risques d'incendie et de panique. Etablissements recevant du public"
df['Contexte'] = df['Source']
for i in range(len(df)):
    for j in [1, 2, 5]:
        if not(pd.isnull(df.loc[i, df.columns[j]])):
            df.loc[i, 'Contexte'] = df.loc[i, 'Contexte'] + str(df.loc[i, df.columns[j]]) + '.x. '

df_erp = df[['Source', 'Contexte', 'Contenu']].copy()

In [33]:
df_erp

,Source,Contexte,Contenu
0,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Classement des établissements§ 1. Les établiss...
1,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Classement des groupements d'établissements ou...
2,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Classement des groupements d'établissements et...
3,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Procédure d'adaptation des règles de sécurité ...
4,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Etablissements comportant des locaux de types ...
...,...,...,...
1172,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Détection automatique d'incendie et système d'...
1173,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Textes applicablesEn complément des dispositio...
1174,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Composition du registre de sécuritéA. - Premiè...
1175,Règlement de sécurité contre les risques d'inc...,Règlement de sécurité contre les risques d'inc...,Liste des matériaux textiles soumis aux intemp...


In [34]:
print(df_ert.shape)
print(df_rsi.shape)
print(df_erp.shape)
print(df_hab.shape)

(299, 3)
(58, 3)
(1177, 3)
(2659, 3)


In [35]:
df_global = pd.concat([df_ert, df_rsi, df_erp, df_hab], axis = 0)
df_global.reset_index(inplace=True, drop=True)

In [36]:
duplicates = df_global['Contenu'].duplicated()
index_to_keep = [not(item) for item in duplicates]
df_global = df_global.loc[index_to_supress, :]
df_global.reset_index(inplace=True, drop=True)

In [37]:
df_global.to_excel('base_de_donnees.xlsx', index=False)